In [34]:
pip install arxiv

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6048 sha256=1d4f6e15c76dc73793feb3f42c965bc832af742be1925f773e6d8f10096e254b
  Stored in directory: /home/ubuntu/.cache/pip/wheels/f0/69/93/a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built sgmllib3k
Note: you may need to restart the kernel to use updated packages.


In [8]:
import urllib.request as libreq
import xmltodict, json
from pprint import pprint

In [19]:

with libreq.urlopen('http://export.arxiv.org/api/query?search_query=all:recsys&start=0&max_results=2&sortBy=relevance&sortOrder=descending') as url:
    r = url.read()
print(r)


b'<?xml version="1.0" encoding="UTF-8"?>\n<feed xmlns="http://www.w3.org/2005/Atom">\n  <link href="http://arxiv.org/api/query?search_query%3Dall%3Arecsys%26id_list%3D%26start%3D0%26max_results%3D2" rel="self" type="application/atom+xml"/>\n  <title type="html">ArXiv Query: search_query=all:recsys&amp;id_list=&amp;start=0&amp;max_results=2</title>\n  <id>http://arxiv.org/api/an+Fj2R0gQtu5A44l3AaQmhOvK8</id>\n  <updated>2024-11-19T00:00:00-05:00</updated>\n  <opensearch:totalResults xmlns:opensearch="http://a9.com/-/spec/opensearch/1.1/">466</opensearch:totalResults>\n  <opensearch:startIndex xmlns:opensearch="http://a9.com/-/spec/opensearch/1.1/">0</opensearch:startIndex>\n  <opensearch:itemsPerPage xmlns:opensearch="http://a9.com/-/spec/opensearch/1.1/">2</opensearch:itemsPerPage>\n  <entry>\n    <id>http://arxiv.org/abs/2207.08713v1</id>\n    <updated>2022-07-13T12:13:00Z</updated>\n    <published>2022-07-13T12:13:00Z</published>\n    <title>The Impact of Feature Quantity on Recommen

In [24]:

outputs = xmltodict.parse(r)

In [2]:
class Paper:
    def __init__(self, url, title, author, published):
        self.url = url
        self.title = title
        self.author = author
        self.published = published

In [3]:
import arxiv

# Construct the default API client.
client = arxiv.Client()

# Search for the 10 most recent articles matching the keyword "quantum."
search = arxiv.Search(
  query = "recsys",
  max_results = 10,
  sort_by = arxiv.SortCriterion.Relevance
)


In [4]:
papers = []
for paper_arxiv in client.results(search):
    # Download the PDF to a specified directory with a custom filename.
    try:
        paper_arxiv.download_pdf(dirpath="/home/ubuntu/Documents/PSP/dump", ) #filename="downloaded-paper.pdf")
        papers.append(Paper(paper_arxiv.entry_id,  paper_arxiv.title, paper_arxiv.authors, paper_arxiv.published,))
    except:
        pass

In [5]:
pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 8.7 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [13]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    # Open the PDF file
    pdf_document = fitz.open(pdf_path)
    text = ""
    
    # Iterate through each page and extract text
    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        text += page.get_text()
    
    return text

pdf_path = '/home/ubuntu/Documents/PSP/dump/1612.00959v1.RecSys_Challenge_2016__job_recommendations_based_on_preselection_of_offers_and_gradient_boosting.pdf'  # Replace with your PDF file path
text = extract_text_from_pdf(pdf_path)



def find_references_section(text):
    # Find the start of the references section
    references_start = text.lower().find("references")
    if references_start == -1:
        references_start = text.lower().find("bibliography")
    
    if references_start == -1:
        raise ValueError("References section not found")
    
    # Extract the references section
    references_text = text[references_start:]
    return references_text

references_text = find_references_section(text)



import re

def extract_paper_names(references_text):
    # Regular expression to match the start of a reference
    reference_pattern = re.compile(r'^\s*\[\d+\]\s+(.*)', re.MULTILINE)
    
    # Find all matches
    matches = reference_pattern.findall(references_text)
    
    # Remove duplicates by converting to a set, then back to a list
    unique_paper_names = list(set(matches))
    
    return unique_paper_names

paper_names = extract_paper_names(references_text)
for name in paper_names:
    print(name)

P. Romov and E. Sokolov. Recsys challenge 2015: Ensemble learning with categorical features. In Proceedings
T. Chen and C. Guestrin. Xgboost: A scalable tree boosting system. In B. Krishnapuram, M. Shah, A. J.
M. Volkovs. Two-stage approach to item recommendation from user sessions. In Proceedings of the 2015


['P. Romov and E. Sokolov. Recsys challenge 2015: Ensemble learning with categorical features. In Proceedings', 'T. Chen and C. Guestrin. Xgboost: A scalable tree boosting system. In B. Krishnapuram, M. Shah, A. J.', 'M. Volkovs. Two-stage approach to item recommendation from user sessions. In Proceedings of the 2015']
